# Análise de Comportamento de Bots Suspeitos

In [1]:
# dependências
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import community

import os
os.chdir("..")

from util.load_graph import load_graph_by_edge, get_driver, get_user_property_keys, get_user_properties, get_property_values, get_property_values_for_users

In [2]:
driver = get_driver()

In [3]:
suspected_bots = pd.read_csv('../data/all_suspected_bots.csv')
suspected_bots

,user_id
0,584b05446c0afec7298ccd5c0a11fa0a
1,e5fd2f4f70b2fbaca3c512672496eafe
2,6dea034b12e1631449450a0df69b752d
3,c923ed78ee5cf2ecbd55ab500f19a5d4
4,9f8f568a201ef009ea0ea2eba71a2d91
...,...
889,618249afda336c0c6943a985bad79c2b
890,75f92f053c96a3486fbed604529be2b5
891,03a05508f9a2899dcd3e689fb47ed900
892,83e8d4d4152aac9baa588ab6697be55d


In [4]:
high_conf_suspected_bots = pd.read_csv('../data/highest_confident_bots.csv')
high_conf_suspected_bots

,user_id
0,b1004f1254518ccb2e5ea5b8dd49b473
1,db60a046ca125348b9cd4dda729b99ec
2,826c407c3fc0646f32523e05bd57b459
3,036dd9c2c985d16f761e93ce7fe79919
4,8d1a3c5b2ba0f29df8a6a5be0bb66191
5,4d825174847c8e2d7db1c77a58e9b725
6,26cd985388f2cf5c0b7918ec6fb6a87f
7,9bc9ea4800037405bcdf572d68631bd3


## Comparação Scores Gerais vs Suspeitos

In [5]:
def compare_property_averages(driver, suspected_bots_df, user_id_col='user_id'):
    sample_user = suspected_bots_df[user_id_col].iloc[0]
    properties = get_user_properties(driver, sample_user)

    results = []

    for prop, val in properties.items():
        if not isinstance(val, (int, float)):
            continue

        all_vals = get_property_values(driver, prop)
        susp_vals = get_property_values_for_users(
            driver, prop, suspected_bots_df[user_id_col]
        )

        if len(all_vals) == 0 or len(susp_vals) == 0:
            continue

        results.append({
            "property": prop,
            "avg_all_users": np.mean(all_vals),
            "avg_suspected_bots": np.mean(susp_vals),
            "ratio_susp_vs_all": np.mean(susp_vals) / np.mean(all_vals)
        })

    df = pd.DataFrame(results).sort_values(
        by="ratio_susp_vs_all", ascending=False
    )

    return df

In [6]:
compare_property_averages(driver, suspected_bots)

,property,avg_all_users,avg_suspected_bots,ratio_susp_vs_all
7,viral_score,0.006145,0.066683,10.851909
3,misinfo_score,0.132189,0.670096,5.069229
5,shares_score,0.522397,1.928357,3.691365
6,synchronicity_score,9.805440,15.342482,1.564691
2,metronome_score,1.484604,1.504869,1.013650
0,content_originality,0.936023,0.776093,0.829138
4,network_diversity,0.557761,0.338266,0.606471
1,content_uniqueness,0.705840,0.292302,0.414120


In [7]:
compare_property_averages(driver, high_conf_suspected_bots)

,property,avg_all_users,avg_suspected_bots,ratio_susp_vs_all
3,misinfo_score,0.132189,1.537492,11.631015
5,shares_score,0.522397,2.771020,5.304435
2,metronome_score,1.484604,1.414798,0.952980
0,content_originality,0.936023,0.748413,0.799567
6,synchronicity_score,9.805440,6.314804,0.644010
4,network_diversity,0.557761,0.192369,0.344896
1,content_uniqueness,0.705840,0.083611,0.118456
7,viral_score,0.006145,0.000000,0.000000


# Encontrar subcomunidades

## Tipos de Similaridade e Arquétipos de Bots

### 1. SYNC_SIMILAR (Intensidade de Coordenação)

**O que mede:**  
Conecta bots que possuem um **Score de Sincronicidade** semelhante (relaciona quantos conteúdos compartilhados com quantos usuários).

**Arquétipos de bots identificados:**

- **Comunidade de alto score:** *Núcleo Central*.  
  Um grupo coeso de bots responsável pela explosão inicial e rápida de mensagens.

- **Comunidade de baixo score:** *Periferia*.  
  Bots que ficam para trás ou participam apenas de ondas menores e específicas.

---

### 2. METRONOME_SIMILAR (Impressão Digital Temporal)

**O que mede:**  
Conecta bots que compartilham o mesmo **Coeficiente de Variação** nos intervalos de postagem.

**Arquétipos de bots identificados:**

- **Comunidade A:** *Relógios*.  
  Bots com variância próxima de zero (extremamente robóticos).

- **Comunidade B:** *Mímicos*.  
  Bots configurados com um algoritmo específico de aleatorização para parecerem humanos (por exemplo, variância maior, porém idêntica entre si).

---

### 3. SHARES_SIMILAR (Perfil de Volume)

**O que mede:**  
Conecta bots com volume total de mensagens semelhante.

**Arquétipos de bots identificados:**

- **Comunidade de alto volume:** *Canhões de Spam*.  
  Projetados para inundar grupos.

- **Comunidade de baixo volume:** *Atiradores de Elite*.  
  Bots que postam raramente.

---

### 4. MISINFO_SIMILAR (Carga Útil: Desinformação)

**O que mede:**  
Conecta bots que dedicam uma proporção semelhante de sua atividade a mensagens com alto score de desinformação.

**Arquétipos de bots identificados:**

- **Comunidade de alto score:** *Especialistas em Desinformação*.  
  Contas criadas especificamente para espalhar narrativas direcionadas.

- **Comunidade de baixo score:** *Amplificadores Genéricos*.  
  Bots que amplificam conteúdo independentemente de sua veracidade.

---

### 5. VIRAL_SIMILAR (Carga Útil: Viralidade)

**O que mede:**  
Conecta bots com taxas de sucesso semelhantes na amplificação de conteúdo viral.

**Arquétipos de bots identificados:**

- **Comunidade de alto score:** *Amplificadores*.  
  Bots projetados para fazer conteúdos existentes virarem tendência.

- **Comunidade de baixo score:**  
  Bots que introduzem conteúdo que não se tornou viral.

---

### 6. CONTENT_ORIGINALITY_SIMILAR (Lógica de Repetição)

**O que mede:**  
Conecta bots com proporções semelhantes de **Textos Únicos / Total de Mensagens**.

**Arquétipos de bots identificados:**

- **Comunidade ~0.1 (Baixa):** *Drones de Copiar e Colar*.  
  Bots que encaminham exatamente o mesmo texto centenas de vezes.

- **Comunidade ~0.9 (Alta):** *Bots Generativos*.  
  Bots que alteram levemente o texto a cada postagem.

---

### 7. CONTENT_UNIQUENESS_SIMILAR (Efeito Câmara de Eco)

**O que mede:**  
Conecta bots com base no quão “mainstream” é o conteúdo que compartilham globalmente.

**Arquétipos de bots identificados:**

- **Comunidade de baixa singularidade:** *Câmara de Eco*.  
  Bots que compartilham apenas o que milhares de outras pessoas já estão compartilhando (reforço de consenso).

- **Comunidade de alta singularidade:** *Injetores*.  
  Bots que introduzem narrativas raras ou novas na rede.

---

### 8. NETWORK_DIVERSITY_SIMILAR (Estratégia de Implantação)

**O que mede:**  
Conecta bots com proporções semelhantes de **Grupos Únicos / Total de Parceiros**.

**Arquétipos de bots identificados:**

- **Comunidade de baixa diversidade:** *Bots Enxame*.  
  Conjunto de bots que atacam o mesmo pequeno grupo de comunidades, sobrecarregando um alvo específico.

- **Comunidade de alta diversidade:** *Bots Ponte*.  
  Bots que atuam como conectores, levando conteúdo de uma comunidade para outros grupos.


In [8]:
def classify_bot_behaviors(driver, suspected_bot_ids, output_dir='../data/visualization'):
    
    os.makedirs(output_dir, exist_ok=True)
    
    similarity_graphs = {
        'SYNC_SIMILAR': 'Coordenação',
        'METRONOME_SIMILAR': 'Variação temporal',
        'SHARES_SIMILAR': 'Compartilhamento',
        'MISINFO_SIMILAR': 'Desinformação',
        'VIRAL_SIMILAR': 'Viral',
        'CONTENT_ORIGINALITY_SIMILAR': 'Originalidade do conteúdo',
        'CONTENT_UNIQUENESS_SIMILAR': 'Singularidade do conteúdo',
        'NETWORK_DIVERSITY_SIMILAR': 'Diversidade da rede'
    }
    
    bot_classifications = {}
    
    for graph_type, behavior_name in similarity_graphs.items():
        print(f"Processando {graph_type}...")
        G = load_graph_by_edge(driver, graph_type)
        
        suspected_nodes = [n for n in G.nodes() if n in suspected_bot_ids]
        G_bots = G.subgraph(suspected_nodes).copy()
        
        if len(G_bots.nodes()) < 2:
            print(f"Pulando {graph_type}: Menos que 2 nós")
            continue
        
        # CONNECTED COMPONENTS
        components = list(nx.connected_components(G_bots))
        print(f"Achou {len(components)} componentes conectados")

        for comp_id, comp in enumerate(components):
            subgraph = G_bots.subgraph(comp)
            density = nx.density(subgraph)
            avg_weight = (
                sum(d['weight'] for _, _, d in subgraph.edges(data=True)) /
                max(subgraph.number_of_edges(), 1)
            )

            for node in comp:
                G_bots.nodes[node]['component_id'] = comp_id
                G_bots.nodes[node]['component_size'] = len(comp)
                G_bots.nodes[node]['component_density'] = round(density, 4)
                G_bots.nodes[node]['component_avg_weight'] = round(avg_weight, 4)
                G_bots.nodes[node]['behavior_type'] = behavior_name

        # LOUVAIN
        louvain_communities = community.louvain_communities(
            G_bots, weight='weight', seed=42
        )
        print(f"Achou {len(louvain_communities)} comunidades Louvain")

        for i, comm in enumerate(louvain_communities):
            for node in comm:
                G_bots.nodes[node]['louvain_community'] = i
                G_bots.nodes[node]['louvain_community_size'] = len(comm)

                if node not in bot_classifications:
                    bot_classifications[node] = {}
                bot_classifications[node][behavior_name] = i

        # K-CORE
        try:
            core_numbers = nx.core_number(G_bots)
            for node, core in core_numbers.items():
                G_bots.nodes[node]['k_core'] = core
        except nx.NetworkXError:
            for node in G_bots.nodes():
                G_bots.nodes[node]['k_core'] = 0
        
        for node in G_bots.nodes():
            G_bots.nodes[node]['degree'] = G_bots.degree(node, weight='weight')
        
        gexf_filename = os.path.join(output_dir, f'{graph_type.lower()}_bot_communities.gexf')
        nx.write_gexf(G_bots, gexf_filename)
        print(f"  -> Salvou {gexf_filename}")
        print()
    
    if bot_classifications:
        df = pd.DataFrame.from_dict(bot_classifications, orient='index')
        df.index.name = 'user_id'
        df = df.reset_index()
        
        csv_filename = os.path.join(output_dir, 'bot_behavior_classifications.csv')
        df.to_csv(csv_filename, index=False)
        print(f"Salvo classificações {csv_filename}")
        
        return df
    else:
        return pd.DataFrame()

In [9]:
classify_bot_behaviors(driver, set(suspected_bots['user_id'].astype(str)))

Processando SYNC_SIMILAR...
Achou 18 componentes conectados
Achou 18 comunidades Louvain
  -> Salvou ../data/visualization/sync_similar_bot_communities.gexf

Processando METRONOME_SIMILAR...
Achou 7 componentes conectados
Achou 13 comunidades Louvain
  -> Salvou ../data/visualization/metronome_similar_bot_communities.gexf

Processando SHARES_SIMILAR...
Achou 8 componentes conectados
Achou 15 comunidades Louvain
  -> Salvou ../data/visualization/shares_similar_bot_communities.gexf

Processando MISINFO_SIMILAR...
Achou 8 componentes conectados
Achou 13 comunidades Louvain
  -> Salvou ../data/visualization/misinfo_similar_bot_communities.gexf

Processando VIRAL_SIMILAR...
Achou 2 componentes conectados
Achou 2 comunidades Louvain
  -> Salvou ../data/visualization/viral_similar_bot_communities.gexf

Processando CONTENT_ORIGINALITY_SIMILAR...
Achou 44 componentes conectados
Achou 44 comunidades Louvain
  -> Salvou ../data/visualization/content_originality_similar_bot_communities.gexf

Proce

,user_id,Coordenação,Variação temporal,Compartilhamento,Desinformação,Singularidade do conteúdo,Diversidade da rede,Originalidade do conteúdo,Viral
0,8f7bd84a25f6cc58c013828e38ba339d,0.0,4.0,11.0,9.0,5.0,9.0,NaN,NaN
1,83e8d4d4152aac9baa588ab6697be55d,0.0,4.0,11.0,8.0,NaN,7.0,NaN,NaN
2,91810e646ca7c95124f3dd7e3961d87a,0.0,8.0,11.0,8.0,86.0,7.0,8.0,NaN
3,826c407c3fc0646f32523e05bd57b459,0.0,4.0,4.0,8.0,12.0,7.0,NaN,NaN
4,0e73d4c78c9a94d2d45341b36820cb1a,0.0,8.0,4.0,11.0,NaN,7.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
889,8a24d97c1cc57b95c22d06d38e096bbb,NaN,NaN,13.0,NaN,37.0,6.0,5.0,NaN
890,ec7094174234f45cfebd9935d9bc0a4a,NaN,NaN,13.0,NaN,80.0,6.0,35.0,NaN
891,668379871f925925b75be1d8aeed1a26,NaN,NaN,13.0,NaN,33.0,3.0,NaN,NaN
892,3505e2a48a426674e839996f69bb41ce,NaN,NaN,13.0,NaN,80.0,13.0,5.0,NaN


In [ ]:
classify_bot_behaviors(driver, set(high_conf_suspected_bots['user_id'].astype(str)))